In [1]:
import sys
sys.path.append("..") # append the directory above where the databese is.
import pandas as pd
import numpy as np
import sqlite3
import time
import numba
%load_ext line_profiler
### disable scientific notation in pandas
pd.set_option('display.float_format', '{:.2f}'.format) ### display up to 2 decimal pts

In [2]:
connection = sqlite3.connect("../db.sqlite3")
cursor = connection.cursor() 
cursor.execute("SELECT * FROM CountryIndicators")  
country_list = cursor.fetchall() # we get the 
np.shape(country_list)
print(country_list[:3]) # okay it has the same format as the table where the rows are tuples in this list

[('ARB', 'SP.ADO.TFRT', 1960, 133.56090740552298), ('ARB', 'SP.POP.DPND', 1960, 87.7976011532547), ('ARB', 'SP.POP.DPND.OL', 1960, 6.634579191565161)]


In [3]:
df = pd.DataFrame(country_list, columns=["CountryCode", "IndicatorCode", "Year", "Value" ])

In [4]:
df_ARB = df[df["CountryCode"] == "ARB"]
print(df_ARB["IndicatorCode"].nunique())
df_one_ind = df_ARB[df["IndicatorCode"] == "SP.ADO.TFRT"][["Year", "Value"]]
df_one_ind = df_one_ind.rename(columns={"Value":"indicator"})

df_another_ind = df_ARB[df["IndicatorCode"] == "SP.POP.DPND"][["Year", "Value"]]
df_another_ind = df_another_ind.rename(columns={"Value":"another_indicator"})
df_merged = df_one_ind.merge(df_another_ind, left_on="Year", right_on="Year")
 #okay this works so
print(df_merged)


629
    Year  indicator  another_indicator
0   1960     133.56              87.80
1   1961     134.16              89.22
2   1962     134.86              90.50
3   1963     134.50              91.60
4   1964     134.10              92.42
5   1965     133.57              92.87
6   1966     132.68              93.73
7   1967     131.67              94.10
8   1968     129.20              94.12
9   1969     126.75              93.96
10  1970     124.40              93.71
11  1971     122.16              93.92
12  1972     120.04              93.90
13  1973     118.11              93.71
14  1974     116.16              93.42
15  1975     114.13              93.08
16  1976     112.01              92.92
17  1977     109.81              92.71
18  1978     106.05              92.45
19  1979     102.36              92.12
20  1980      98.75              91.68
21  1981      95.26              91.48
22  1982      91.81              91.11
23  1983      88.01              90.62
24  1984      84.22  

In [ ]:

df_ARB[df_ARB["IndicatorCode"] == "AG.AGR.TRAC.NO"]
#for indicator in df_ARB["IndicatorCode"]:
    #print(indicator)

In [9]:
#def uniques(df, indicator):
    #""" returns true or false if the indicator is in every year"""
    #indicator_range = np.array(df[df["IndicatorCode"] == indicator]["Year"])
    #year_range = np.array(range(df["Year"].min(), df["Year"].max() + 1))
    #if len(year_range) != len(indicator_range):
     #   return False
    #return True
def create_total_df(df):
    df_final = pd.DataFrame(df["Year"])
   # print(df_final)
    #print(df["IndicatorCode"].nunique())
    for indicator in df["IndicatorCode"].unique():
        #print(f"{indicator}")
        #if uniques(df, indicator):
        df_tmp = df[df["IndicatorCode"] == indicator][["Year", "Value"]]
        df_tmp = df_tmp.rename(columns={"Value":indicator})
        df_final = pd.merge(df_final, df_tmp, "Year", how="outer")
        print(indicator)
    return df_final

#df_s = []
%lprun -f create_total_df create_total_df(df_ARB[df_ARB["Year"] == 2010])
#ncountrys = df["CountryCode"].nunique()
#count = 0
#for country in df["CountryCode"].unique():
 #   s = time.time()
  #  df_s.append(create_total_df(df[df["CountryCode"] == country]))
   # count += 1
    #print(country)
    #print(f"ETA: {(time.time() - s) *(ncountrys - 1)/60} min")


TypeError: merge() got multiple values for argument 'how'

In [ ]:
def create_country_df(df_country):
    country_dict = {indicator:[] for indicator in df_country["IndicatorCode"].unique()}
    country_dict["Year"] = list(df_country["Year"].unique())
    for year in df_country["Year"].unique():
        for indicator in df_country["IndicatorCode"].unique():
            # this will output the indicator value for a given year
            df_tmp = df_country[(df_country["IndicatorCode"] == indicator) & (df_country["Year"] == year)]["Value"] 
            # if there is no indicator value for that year, then we add NA                              
            if df_tmp.empty:
                country_dict[indicator].append("NA")
            else:
                country_dict[indicator].append(df_tmp.to_list()[0]) # fix this is really ugly!!
        print(year)
    return pd.DataFrame(country_dict)
%lprun -f create_country_df create_country_df(df_ARB)

In [ ]:
def create_country_df(df_country):
    country_dict = {indicator:[] for indicator in df_country["IndicatorCode"].unique()}
    country_dict["Year"] = list(df_country["Year"].unique())
    for year in df_country["Year"].unique():
        for indicator in df_country["IndicatorCode"].unique():
            # this will output the indicator value for a given year
            df_tmp = df_country[(df_country["IndicatorCode"] == indicator) & (df_country["Year"] == year)]["Value"] 
            # if there is no indicator value for that year, then we add NA                              
            if df_tmp.empty:
                country_dict[indicator].append("NA")
            else:
                country_dict[indicator].append(df_tmp.to_list()[0]) # fix this is really ugly!!
        print(year)
    return pd.DataFrame(country_dict)